In [3]:
# 1) Load & inspect the dataset structure
from datasets import load_dataset
import pandas as pd

# pick one split from: by_polishing, from_title, from_title_and_content
split_name = "by_polishing"

dataset = load_dataset("KFUPM-JRCAI/arabic-generated-abstracts")
df = dataset[split_name].to_pandas()

print("=== RAW SPLIT STRUCTURE ===")
df.info()

print("=== snap of data ===")
df.head()


=== RAW SPLIT STRUCTURE ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2851 entries, 0 to 2850
Data columns (total 5 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   original_abstract          2851 non-null   object
 1   allam_generated_abstract   2851 non-null   object
 2   jais_generated_abstract    2851 non-null   object
 3   llama_generated_abstract   2851 non-null   object
 4   openai_generated_abstract  2851 non-null   object
dtypes: object(5)
memory usage: 111.5+ KB
=== snap of data ===


,original_abstract,allam_generated_abstract,jais_generated_abstract,llama_generated_abstract,openai_generated_abstract
0,كثيرا ما ارتبطت المصادر التاريخية في الأندلس خ...,يتناول هذا البحث موضوع التعليم بين النساء الأن...,تدرس هذه الدراسة دور المرأة في التعليم في الأن...,يُقدم هذا البحث دراسة شاملة حول حالة التعليم ع...,صور نظام التعليم عند المرأة الأندلسية تستند إل...
1,يعد العامل الثقافي احد ابرز الاسباب التي يعزى ...,يتناول هذا البحث دراسة انهيار دولة الموحدين من...,كان العامل الثقافي من بين الأسباب الرئيسية الت...,يعد العامل الثقافي أحد أبرز الأسباب التي يعزى ...,انهيار دولة الموحدين يعود بشكل كبير للعوامل ال...
2,شكلت تلك الجهود والمساعي الرائدة التي قام بها ...,هدفت هذه الدراسة إلى تسليط الضوء على جهود قادة...,تدرس هذه الدراسة جهود قادة الثورة الجزائرية خل...,شكلت الجهود التي بذلها قادة الثورة الجزائرية خ...,جهود قادة الثورة الجزائرية خلال المرحلة الأولى...
3,يقوم المقال على اشكالية الضرائب الغير شرعية في...,يتناول هذا البحث إشكالية الضرائب غير الشرعية ف...,تدرس المقالة مشكلة الضرائب غير الشرعية في مراح...,يقوم البحث على دراسة الضرائب غير الشرعية في دو...,المقال يناقش قضية الضرائب غير الشرعية في دولتي...
4,تتفق المصادر التاريخية المتوفرة حول موضوع تطور...,تتناول هذه الدراسة حركة الانتصار للحريات الديم...,حركة انتصار الحريات الديمقراطية (MTLD)، وهي حر...,تُظهر المصادر التاريخية المتاحة حول تطور الحرك...,حركة انتصار الحريات الديمقراطية (MTLD) في الجز...


In [5]:
#2) Build a labeled dataframe (human vs AI)
# columns present in this dataset
AI_COLS = ["allam_generated_abstract", "jais_generated_abstract",
           "llama_generated_abstract", "openai_generated_abstract"]

# human
human = pd.DataFrame({
    "text": df["original_abstract"],
    "label": "human",
    "source": "original"
})

# AI (stack all generators)
ai_frames = []
for c in AI_COLS:
    ai_frames.append(pd.DataFrame({
        "text": df[c],
        "label": "AI",
        "source": c.replace("_generated_abstract", "")
    }))

full_df = pd.concat([human] + ai_frames, ignore_index=True)

print("=== LABELED DATA STRUCTURE ===")
full_df.info()
full_df.head()


=== LABELED DATA STRUCTURE ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14255 entries, 0 to 14254
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    14255 non-null  object
 1   label   14255 non-null  object
 2   source  14255 non-null  object
dtypes: object(3)
memory usage: 334.2+ KB


,text,label,source
0,كثيرا ما ارتبطت المصادر التاريخية في الأندلس خ...,human,original
1,يعد العامل الثقافي احد ابرز الاسباب التي يعزى ...,human,original
2,شكلت تلك الجهود والمساعي الرائدة التي قام بها ...,human,original
3,يقوم المقال على اشكالية الضرائب الغير شرعية في...,human,original
4,تتفق المصادر التاريخية المتوفرة حول موضوع تطور...,human,original


In [8]:
#3) Distribution of the target variable 
print("=== LABEL DISTRIBUTION (counts) ===")
print(full_df["label"].value_counts())

print("\n=== LABEL DISTRIBUTION (%) ===")
print((full_df["label"].value_counts(normalize=True) * 100).round(2))

print("\n=== SOURCE DISTRIBUTION ===")
print(full_df["source"].value_counts())


=== LABEL DISTRIBUTION (counts) ===
label
AI       11404
human     2851
Name: count, dtype: int64

=== LABEL DISTRIBUTION (%) ===
label
AI       80.0
human    20.0
Name: proportion, dtype: float64

=== SOURCE DISTRIBUTION ===
source
original    2851
allam       2851
jais        2851
llama       2851
openai      2851
Name: count, dtype: int64


In [9]:
# 4) Data quality checks (missing, duplicates, inconsistencies)
# trim whitespace for consistency
full_df["text"] = full_df["text"].astype(str).str.strip()

print("=== MISSING VALUES PER COLUMN ===")
print(full_df.isna().sum())

print("\n=== EMPTY TEXT ROWS (after strip) ===")
empty_count = (full_df["text"] == "").sum()
print(empty_count)

print("\n=== DUPLICATES (exact text duplicates) ===")
dup_count = full_df.duplicated(subset=["text"]).sum()
print(dup_count)

print("\n=== BASIC LENGTH STATS (chars) ===")
lengths = full_df["text"].str.len()
print(lengths.describe().astype(int))

# (optional) quickly preview any suspicious rows
print("\n=== SAMPLE EMPTY OR VERY SHORT TEXTS (<= 10 chars) ===")
suspicious = full_df[full_df["text"].str.len() <= 10].head(10)
suspicious


=== MISSING VALUES PER COLUMN ===
text      0
label     0
source    0
dtype: int64

=== EMPTY TEXT ROWS (after strip) ===
0

=== DUPLICATES (exact text duplicates) ===
16

=== BASIC LENGTH STATS (chars) ===
count    14255
mean       713
std        305
min        170
25%        511
50%        654
75%        922
max      18459
Name: text, dtype: int64

=== SAMPLE EMPTY OR VERY SHORT TEXTS (<= 10 chars) ===


,text,label,source
